In [1]:
import pandas as pd
import math
import get_df

In [2]:
df = get_df.get_df(nrows=80)[["match_id", "Svr", "1stIn", "2ndIn", "isSvrWinner"]]
df

,match_id,Svr,1stIn,2ndIn,isSvrWinner
0,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,0,0.0,0
1,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,1,NaN,1
2,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,0,1.0,0
3,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,1,NaN,1
4,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,1,NaN,1
...,...,...,...,...,...
75,20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cri...,2,1,NaN,1
76,20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cri...,1,1,NaN,0
77,20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cri...,1,1,NaN,1
78,20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cri...,1,1,NaN,0


In [3]:
df_fs = df[df["1stIn"] == 1]
df_ss = df[df["2ndIn"] == 1]

x1=df_fs.shape[0]/df.shape[0] #1st in / (number of games=number of 1st serves)

y1=df_fs[df_fs["isSvrWinner"]==1].shape[0]/df_fs.shape[0] #(number of games won on first serve/ (number of 1st in) )
w1=y1*x1

x2=df_ss.shape[0]/(df.shape[0]-df_fs.shape[0])
y2=df_ss[df_ss["isSvrWinner"]==1].shape[0]/df_ss.shape[0]
w2=y2*x2

p=w1+(1-x1)*w2

x1_var = math.sqrt(x1*(1-x1)/df.shape[0])
x2_var = math.sqrt(x2*(1-x2)/df.shape[0])
y1_var = math.sqrt(y1*(1-y1)/df_fs.shape[0])
y2_var = math.sqrt(y2*(1-y2)/df_fs.shape[0])
# w2_var=
print("x1:", x1)
print("y1:", y1)
print("w1:", w1)
print("x2:", x2)
print("y2:", y2)
print("w2:", w2)
print("P:", p)
print("real P:", df[df["isSvrWinner"]==1].shape[0]/df.shape[0])

x1: 0.6625
y1: 0.6745283018867925
w1: 0.446875
x2: 0.8703703703703703
y2: 0.46808510638297873
w2: 0.4074074074074074
P: 0.5843750000000001
real P: 0.584375


In [4]:
#Demonstration of using the 1st and 2nd serve strategy twice
p_1s=w1+(1-x1)*w1
p_2s=w2+(1-x2)*w2
print("P using strategy 1:", p_1s) # Probability of winning using second serve strategy twice
print("P using strategy 2:", p_2s) # Probability of winning using first serve strategy twice

P using strategy 1: 0.5976953125000001
P using strategy 2: 0.46021947873799723


In [5]:
# average of values by game

df_by_match = df.groupby(["match_id", "Svr"])
serve_stats = df_by_match.agg(x_1=("1stIn", "mean"), x_2=("2ndIn", "mean"))
serve_stats["y_1"] = df_by_match.apply(lambda x: x[x["1stIn"]==1]["isSvrWinner"].mean())
serve_stats["y_2"] = df_by_match.apply(lambda x: x[x["2ndIn"]==1]["isSvrWinner"].mean())

serve_stats["w_1"] = serve_stats["x_1"]*serve_stats["y_1"]
serve_stats["w_2"] = serve_stats["x_2"]*serve_stats["y_2"]

serve_stats



x_1       x_2  \
match_id                                           Svr                       
20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cris... 1    0.608696  0.944444   
                                                   2    0.823529  0.833333   
20161127-M-Davis_Cup_World_Group_F-RR-Marin_Cil... 1    0.666667  1.000000   
                                                   2    0.656250  1.000000   
20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pabl... 1    0.480769  0.740741   
                                                   2    0.750000  0.857143   
20221201-W-Andorra-R16-Cristina_Bucsa-Kateryna_... 1    0.605263  0.866667   
                                                   2    0.809524  0.750000   

                                                             y_1       y_2  \
match_id                                           Svr                       
20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cris... 1    0.500000  0.470588   
                                                   2    0.642857  0.800000   
20161127-M-Davis_Cup_World_Group_F-RR-Marin_Cil... 1    0.812500  0.375000   
                                                   2    0.761905  0.545455   
20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pabl... 1    0.800000  0.350000   
                                                   2    0.666667  0.833333   
20221201-W-Andorra-R16-Cristina_Bucsa-Kateryna_... 1    0.695652  0.461538   
                                                   2    0.558824  0.333333   

                                                             w_1       w_2  
match_id                                           Svr                      
20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cris... 1    0.304348  0.444444  
                                                   2    0.529412  0.666667  
20161127-M-Davis_Cup_World_Group_F-RR-Marin_Cil... 1    0.541667  0.375000  
                                                   2    0.500000  0.545455  
20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pabl... 1    0.384615  0.259259  
                                                   2    0.500000  0.714286  
20221201-W-Andorra-R16-Cristina_Bucsa-Kateryna_... 1    0.421053  0.400000  
                                                   2    0.452381  0.250000

In [6]:
points_db = pd.concat((serve_stats[["x_1", "y_1"]].rename(columns={"x_1": "x", "y_1": "y"}),
                       serve_stats[["x_2", "y_2"]].rename(columns={"x_2": "x", "y_2": "y"}),
                       ),
                      keys=["srv_1", "srv_2"],
                      copy=False)

points_db

x  \
      match_id                                           Svr             
srv_1 20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cris... 1    0.608696   
                                                         2    0.823529   
      20161127-M-Davis_Cup_World_Group_F-RR-Marin_Cil... 1    0.666667   
                                                         2    0.656250   
      20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pabl... 1    0.480769   
                                                         2    0.750000   
      20221201-W-Andorra-R16-Cristina_Bucsa-Kateryna_... 1    0.605263   
                                                         2    0.809524   
srv_2 20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cris... 1    0.944444   
                                                         2    0.833333   
      20161127-M-Davis_Cup_World_Group_F-RR-Marin_Cil... 1    1.000000   
                                                         2    1.000000   
      20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pabl... 1    0.740741   
                                                         2    0.857143   
      20221201-W-Andorra-R16-Cristina_Bucsa-Kateryna_... 1    0.866667   
                                                         2    0.750000   

                                                                     y  
      match_id                                           Svr            
srv_1 20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cris... 1    0.500000  
                                                         2    0.642857  
      20161127-M-Davis_Cup_World_Group_F-RR-Marin_Cil... 1    0.812500  
                                                         2    0.761905  
      20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pabl... 1    0.800000  
                                                         2    0.666667  
      20221201-W-Andorra-R16-Cristina_Bucsa-Kateryna_... 1    0.695652  
                                                         2    0.558824  
srv_2 20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cris... 1    0.470588  
                                                         2    0.800000  
      20161127-M-Davis_Cup_World_Group_F-RR-Marin_Cil... 1    0.375000  
                                                         2    0.545455  
      20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pabl... 1    0.350000  
                                                         2    0.833333  
      20221201-W-Andorra-R16-Cristina_Bucsa-Kateryna_... 1    0.461538  
                                                         2    0.333333